# IaC: Upload csv files to AWS S3 Bucker 

In [4]:
import logging
import boto3
from botocore.exceptions import ClientError
import os

In [3]:
# Retrieve the list of existing buckets
s3 = boto3.client('s3')
response = s3.list_buckets()

# Output the bucket names
print('Existing buckets:')
for bucket in response['Buckets']:
    print(f'  {bucket["Name"]}')

Existing buckets:
  qlik-subscribers


In [14]:
# Set directory for files to upload
directory = r'C:\Users\MartinBirkAndreasen\Desktop\Jazz\QlikWorld\subscription\to_s3'

# Set bucket name
bucket = 'qlik-subscribers'

In [5]:
def upload_file(file_name, bucket, object_name=None):
    """Upload a file to an S3 bucket

    :param file_name: File to upload
    :param bucket: Bucket to upload to
    :param object_name: S3 object name. If not specified then file_name is used
    :return: True if file was uploaded, else False
    """

    # If S3 object_name was not specified, use file_name
    if object_name is None:
        object_name = os.path.basename(file_name)

    # Upload the file
    s3_client = boto3.client('s3')
    try:
        response = s3_client.upload_file(file_name, bucket, object_name)
    except ClientError as e:
        logging.error(e)
        return False
    return True

In [13]:
# Upload all files in directory

for filename in os.listdir(directory):
    f = os.path.join(directory, filename)
    print(f)
    upload_file(f, bucket, object_name=None)

C:\Users\MartinBirkAndreasen\Desktop\Jazz\QlikWorld\subscription\to_s3\customer_cases.csv
C:\Users\MartinBirkAndreasen\Desktop\Jazz\QlikWorld\subscription\to_s3\dim_customer.csv
C:\Users\MartinBirkAndreasen\Desktop\Jazz\QlikWorld\subscription\to_s3\dim_geo.csv
C:\Users\MartinBirkAndreasen\Desktop\Jazz\QlikWorld\subscription\to_s3\fact_active.csv
C:\Users\MartinBirkAndreasen\Desktop\Jazz\QlikWorld\subscription\to_s3\fact_product.csv
C:\Users\MartinBirkAndreasen\Desktop\Jazz\QlikWorld\subscription\to_s3\product_info.csv
